In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import re
import numpy as np
import requests

In [2]:
df_school = pd.read_csv('./datasets/nswschools.csv')
df_suburbs = pd.read_csv('./datasets/SydneyMetro_Postcodes.csv')

In [ ]:
# print (df_school.head())
# print(df_suburbs.head())
# print (df_school.columns)
# print (df_suburbs.columns)
# df_school.head(3)
# df_school.ASGS_remoteness.unique()
print (df_school[df_school['school_name'].str.contains('Bert')].T)
#df_school.aecg_region.value_counts
df_school.head()

In [36]:
#setup driver and open the realestate.com.au website
def open_website(chrome_driver, website, assert_text):
    driver = webdriver.Chrome(executable_path="chrome_driver")
    driver.get(website)
    assert assert_text in driver.title
    return driver

# Request the resource from google maps
def get_geocodes(addr):
    result = requests.get("https://maps.googleapis.com/maps/api/geocode/json?address=" + addr.replace(" ", "+"))
    google_result = result.json()
    # Loop through results and display lat, lon values for reverse geocode
    for item in google_result['results']:
        location= item['geometry']['location']
    return location

#scrape individual page
def scrape_page(html):
    address = []
    latitude = []
    longitude = []

    bedroom = []
    bathroom = []
    carpark = []

    price= []

    html = driver.page_source
    html = BeautifulSoup(html, 'lxml')
    
    for itm in html.find_all('p', {'class':'priceText'}):
        print (itm.text)
        if itm.text.find('$')>=0:
            if itm.text.find(' to ') >= 0:
                prc = re.findall(r'\d+(?:,\d+)?',itm.text)
                prc = [x.replace(',','') for x in prc]
                price.append(np.mean(list(map(int, prc))))
            else:
                prc = re.findall(r'\d+(?:,\d+)?',itm.text)
                if len(prc) != 0:
                    prc = [x.replace(',','') for x in prc]
                    price.append(int(prc[0]))
                else:
                    price.append(0)
        else:
            price.append(0)

    for itm in html.find_all('a', {'class':'name'}):
        address_str = itm.text
        if address_str.find('available on request') == -1:
            geo_codes = get_geocodes (itm.text)
            latitude.append(geo_codes['lat'])
            longitude.append(geo_codes['lng'])
            address.append(itm.text)  
        else:
            latitude.append('NA')
            longitude.append('NA')
            address.append('NA')  
            
    for itm in html.find_all('dl', {'class':'rui-property-features rui-clearfix'}):
        rooms = re.findall('\d+', itm.text)
        rooms = list(map(int, rooms))
        rooms = rooms+[0]*(3-len(rooms))
        bedroom.append (rooms[0])
        bathroom.append(rooms[1])
        carpark.append(rooms[2])
    result = {'address': address,'bedroom': bedroom,'bathroom': bathroom, 'carpark':carpark, 'price': price, 'latitude': latitude, 'longitude': longitude}
    return result

In [30]:
def main(driver):
    i = 0
    #create empty "house dataframe" and keep appending as you scrape pages and fetch data
    cols  = ['address','bedroom','bathroom', 'carpark', 'price', 'latitude', 'longitude']
    house_df  = pd.DataFrame(columns=cols)
    
    #keeping 10 suburbs for now .... should be changed to len(df_suburb)
    
    while i <= 10:
        elem = driver.find_element_by_name("where")
        elem.clear()
        print (df_suburbs.suburb.iloc[i] + ", " + df_suburbs.state.iloc[i] + " " + str(df_suburbs.postcode.iloc[i]))
        print (i)
        elem.send_keys(df_suburbs.suburb.iloc[i] + ", " + df_suburbs.state.iloc[i] + " " + str(df_suburbs.postcode.iloc[i]))

        elem.send_keys(Keys.RETURN)
        assert "No results found." not in driver.page_source
        result = scrape_page(driver)
        new_df = pd.DataFrame.from_dict(result)
        house_df = house_df.append(new_df, ignore_index = True)
        #clear result
        result = {}
        #check if Next Page exists for this suburb
        #print (driver.find_element_by_partial_link_text ("Next"))
        while driver.find_element_by_partial_link_text ("Next") != '':
            driver.find_element_by_partial_link_text ("Next").click()
            result = scrape_page(driver)
            new_df = pd.DataFrame.from_dict(result)
            house_df = house_df.append(new_df, ignore_index = True)
            #clear result
            result = {}
        i +=1
    return house_df

In [31]:
driver = open_website("./chromedriver/chromedriver","Real Estate", "https://www.realestate.com.au/rent")

In [35]:
#house_df = main(driver)

#print (driver.find_element_by_partial_link_text ("Next"))

In [69]:
driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver")
driver.get("http://www.realestate.com.au/rent/in-sydney+city%3b+nsw+2000/list-1")
result = scrape_page(driver)
print (result)
# update_df (result, house_df)
# new_df = pd.DataFrame.from_dict(result)
# house_df = house_df.append(new_df, ignore_index = True)


$1,300 per week
$680 PW
$1795 PW
$1,650 per week
$5,800 per week
$620
$840
$725
$850.00
$705pw
$760 per week
$600 per week
$795pw
$560
$895pw
$695pw
$730
$1,800 per week
$690
$775
$1,650 per week
$750 per week
{'address': ['NA', '85-87 Kent Street, Sydney, NSW 2000', '161 Kent Street, Sydney, NSW 2000', 'NA', '28 B / 171 Gloucester Street, Sydney, NSW 2000', '45 Shelley Street, Sydney, NSW 2000', '708/168 Kent Street, Sydney, NSW 2000', '602/5 York Street, Sydney, NSW 2000', '305/45 Shelley Street, Sydney, NSW 2000', '85-87 Kent Street, Sydney, NSW 2000', '712/168 Kent Street, Sydney, NSW 2000', '37-43 King Street, Sydney, NSW 2000', '22  Sir John Young Crescent, Sydney, NSW 2000', '1 Hosking Place, Sydney, NSW 2000', '87 Cathedral Street, Sydney, NSW 2000', '85-87 Kent Street, Sydney, NSW 2000', '98 Gloucester Street, Sydney, NSW 2000', '25 Barangaroo Ave, Sydney, NSW 2000', '1 Hosking Place, Sydney, NSW 2000', '1307/168 Kent Street, Sydney, NSW 2000', 'NA', 'B301/6-8  Shout Ridge'], 

In [81]:
#mydict = {'address': ['5303/91 Liverpool Street, Sydney, NSW 2000', '1303A/148 Elizabeth Street, Sydney, NSW 2000', '1103/61 Macquarie Street, Sydney, NSW 2000', '38 Bridge Street, Sydney, NSW 2000', '361 Kent Street, Sydney, NSW 2000', '3612/91 Liverpool Street, Sydney, NSW 2000', '88/361 Kent Street, Sydney, NSW 2000', '45 Shelley Street, Sydney, NSW 2000', 'CBD Penthouse for RENT, Sydney, NSW 2000', '103/38 Hickson Road, Millers Point, NSW 2000', '201/19 Barangaroo Avenue, Barangaroo, NSW 2000', '2911/91-95 Liverpool St, Sydney, NSW 2000', 'NA', 'NA', 'NA', '298 Sussex Street, Sydney, NSW 2000', '2707/187 Liverpool Street, Sydney, NSW 2000', '168 Kent Street, Sydney, NSW 2000', '37-43 King Street, Sydney, NSW 2000', '302/21 Alberta Street, Sydney, NSW 2000', 'NA', 'NA'], 'bedroom': [2, 1, 2, 1, 1, 1, 2, 1, 3, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1], 'bathroom': [2, 1, 2, 0, 0, 1, 2, 1, 4, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1], 'carpark': [1, 0, 1, 0, 0, 1, 1, 0, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 2, 1], 'price': [1250, 575, 2350, 545, 595, 820, 920, 795, 1800, 1200, 0, 750, 900, 1250, 840, 785, 1020, 725, 700, 620, 1300, 700], 'latitude': [-33.877274, -33.8777167, -33.8608496, -33.8632738, -33.8692736, -33.877274, -33.8692736, -33.8681676, -33.876613, -33.8605081, -33.864252, -33.877274, 'NA', 'NA', 'NA', -33.8744294, -33.8770938, -33.8622019, -33.8689533, -33.8779375, 'NA', 'NA'], 'longitude': [151.2066911, 151.2098177, 151.2129165, 151.2106721, 151.2043244, 151.2066911, 151.2043244, 151.2022712, 151.206883, 151.2026932, 151.2015689, 151.2066911, 'NA', 'NA', 'NA', 151.204579, 151.2115025, 151.203817, 151.2042688, 151.2105307, 'NA', 'NA']}
# for k,v in result.items():
#     print (len(v))
# type(mydict)
# mydict.keys['address']
print (result['price'])
print(result['address'])

[1300, 680, 1795, 1650, 5800, 620, 840, 725, 850, 705, 760, 600, 795, 560, 895, 695, 730, 1800, 690, 775, 1650, 750]
['NA', '85-87 Kent Street, Sydney, NSW 2000', '161 Kent Street, Sydney, NSW 2000', 'NA', '28 B / 171 Gloucester Street, Sydney, NSW 2000', '45 Shelley Street, Sydney, NSW 2000', '708/168 Kent Street, Sydney, NSW 2000', '602/5 York Street, Sydney, NSW 2000', '305/45 Shelley Street, Sydney, NSW 2000', '85-87 Kent Street, Sydney, NSW 2000', '712/168 Kent Street, Sydney, NSW 2000', '37-43 King Street, Sydney, NSW 2000', '22  Sir John Young Crescent, Sydney, NSW 2000', '1 Hosking Place, Sydney, NSW 2000', '87 Cathedral Street, Sydney, NSW 2000', '85-87 Kent Street, Sydney, NSW 2000', '98 Gloucester Street, Sydney, NSW 2000', '25 Barangaroo Ave, Sydney, NSW 2000', '1 Hosking Place, Sydney, NSW 2000', '1307/168 Kent Street, Sydney, NSW 2000', 'NA', 'B301/6-8  Shout Ridge']


In [34]:
house_df.to_csv('./scraped.csv', mode='a', index=False)


NameError: name 'house_df' is not defined

In [19]:
df_suburbs.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 3 columns):
suburb      202 non-null object
state       202 non-null object
postcode    202 non-null int64
dtypes: int64(1), object(2)
memory usage: 4.8+ KB


In [216]:
len( ['504/720 George Street, Haymarket, NSW 2000', '298 Sussex Street, Sydney, NSW 2000', '308 Pitt Street, Sydney, NSW 2000', '21A Hickson Road, Walsh Bay, NSW 2000', '569 George Street, Sydney, NSW 2000', '2201/91 Liverpool St, Sydney, NSW 2000', 'Level 6/336 Sussex Street, Sydney, NSW 2000', '38 Kent Street, Millers Point, NSW 2000', '106 King Street, Sydney, NSW 2000', '3603/343 Pitt Street, Sydney, NSW 2000', '226/398-408 Pitt Street, Sydney, NSW 2000', 'Level 18/28 Harbour Street, Sydney, NSW 2000', 'Level 8/178 Thomas Street, Sydney, NSW 2000', 'CAR SPACE/2 Bond Street, Sydney, NSW 2000', '612B/317-321 Castlreagh Street, Sydney, NSW 2000', '2 Hay Street, Haymarket, NSW 2000', '507/50 Murray Street, Sydney, NSW 2000', '2-8 Dixon Street, Sydney, NSW 2000', 'NA', '411/27 Park Street, Sydney, NSW 2000', 'NA', 'NA'])
len([1, 660, 1, 1, 900, 700, 850, 1, 1, 860, 1, 850, 100, 440, 1, 850, 480, 1350, 550, 700, 700])
len([2, 1, 3, 3, 2, 1, 2, 3, 2, 2, 3, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1])
len([550, 1300, 1000, 660, 1500, 1600, 900, 700, 850, 1200, 1600, 860, 1545, 850, 100, 440, 1150, 850, 480, 815.0, 900])


21

In [33]:
!rm './scraped.csv'

In [212]:
l = re.findall(pattern=r'\d+(?:,\d+)?', string = '$100,000 to $100,500')
l = [x.replace(',','') for x in l]
print (l)

['100000', '100500']


In [43]:
html = driver.page_source
html = BeautifulSoup(html, 'lxml')
print (html.find_all('dl', {'class':'rui-property-features rui-clearfix'}))

[<dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-car"><span clas

<dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>2</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>1</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>2</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>3</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>, <dl class="rui-property-features rui-clearfix"><dt class="rui-icon rui-icon-bed"><span class="rui-visuallyhidden">Bedrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-bath"><span class="rui-visuallyhidden">Bathrooms</span></dt> <dd>2</dd> <dt class="rui-icon rui-icon-car"><span class="rui-visuallyhidden">Car Spaces</span></dt> <dd>1</dd></dl>

In [13]:
price = []
mystr = ['$550', '$600 to $1000', '$srikanta', '$660', 'contact agent', '$1600 per week']
for string in mystr:
    if string.find('$') >= 0:
        if string.find(' to ') >= 0:
            prc = re.findall(r'\d+(?:,\d+)?',string)
            prc = [x.replace(',','') for x in prc]
            price.append(np.mean(list(map(int, prc))))
        else:
            prc = re.findall(r'\d+(?:,\d+)?', string)
            if len(prc) != 0:
                prc = [x.replace(',', '') for x in prc]
                price.append(int(prc[0]))
            else:
                price.append(0)
    else:
        price.append(0)

print (price)

[550, 800.0, 0, 660, 0, 1600]
